In [79]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import requests
from io import BytesIO
from PIL import Image
from datetime import datetime
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# Define the year and load the data
YEAR = 2024
url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{YEAR}.csv.gz'
data = pd.read_csv(url, compression='gzip', low_memory=False)

# Set options for displaying dataframes
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)

# Filter for plays where scramble = 1
scramble_data = data[data['qb_scramble'] == 1]
qb_list = ["J.Hurts", "J.Allen", "J.Daniels", "P.Mahomes"]

# Filter for scrambles where the rusher_player_name is in the qb_list
scramble_data = scramble_data[scramble_data['rusher_player_name'].isin(qb_list)]
# Calculate EPA/Play for each quarterback
epa_per_scramble = (
    scramble_data.groupby('rusher_player_name')['epa']
    .mean()
    .reset_index()
    .rename(columns={'rusher_player_name': 'QB', 'epa': 'EPA_per_Scramble'})
    .sort_values(by='EPA_per_Scramble', ascending=False)
)

# Display the resulting dataframe
print(epa_per_scramble)


          QB  EPA_per_Scramble
3  P.Mahomes          0.890219
2    J.Hurts          0.809657
0    J.Allen          0.740070
1  J.Daniels          0.665469


In [ ]:
import pandas as pd
import plotly.express as px
from io import BytesIO
from PIL import Image
import requests
import base64

START_YEAR = 2020
END_YEAR = 2024
years = range(START_YEAR, END_YEAR + 1)
all_seasons_penalty_data = []

for YEAR in years:
    print(f"Processing data for the {YEAR} season...")
    url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{YEAR}.csv.gz'
    data = pd.read_csv(url, compression='gzip', low_memory=False)
    
    penalty_weeks_data = data[
        (data['penalty'].notna()) & 
        (data['week'].isin([19, 20, 21, 22]))
    ]
    
    penalty_columns = [col for col in data.columns if 'penalty' in col.lower()]
    penalty_columns.append('wpa')  
    penalty_weeks_data = penalty_weeks_data[penalty_columns + ['penalty_team']]
    
    penalty_weeks_data['season'] = YEAR
    
    all_seasons_penalty_data.append(penalty_weeks_data)

all_seasons_penalty_data = pd.concat(all_seasons_penalty_data, ignore_index=True)

all_seasons_penalty_data = all_seasons_penalty_data.loc[:, ~all_seasons_penalty_data.columns.duplicated()]

all_seasons_penalty_data = all_seasons_penalty_data.dropna(subset=['penalty_team'])

all_seasons_penalty_data['penalty_team'] = all_seasons_penalty_data['penalty_team'].astype(str).str.strip()

penalty_wpa_by_team_season = all_seasons_penalty_data.groupby(['penalty_team', 'season'], as_index=False).agg({'wpa': 'sum'})
penalty_wpa_by_team_season['wpa'] = penalty_wpa_by_team_season['wpa'] * 100  

nfl_teams = [
    {"team": "ARI", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ari.png&h=500&w=500","hex": "#97233F"},
    {"team": "ATL", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/atl.png&h=500&w=500","hex": "#a71930"},
    {"team": "BAL", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/bal.png&h=500&w=500","hex": "#241773"},
    {"team": "BUF", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/buf.png&h=500&w=500","hex": "#00338D"},
    {"team": "CAR", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/car.png&h=500&w=500","hex": "#0085CA"},
    {"team": "CHI", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/chi.png&h=500&w=500","hex": "#0B162A"},
    {"team": "CIN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/cin.png&h=500&w=500","hex": "#fb4f14"},
    {"team": "CLE", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/cle.png&h=500&w=500","hex": "#311D00"},
    {"team": "DAL", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/dal.png&h=500&w=500","hex": "#041E42"},
    {"team": "DEN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/den.png&h=500&w=500","hex": "#FB4F14"},
    {"team": "DET", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/det.png&h=500&w=500","hex": "#0076b6"},
    {"team": "GB", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/gb.png&h=500&w=500","hex": "#203731"},
    {"team": "HOU", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/hou.png&h=500&w=500","hex": "#03202f"},
    {"team": "IND", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ind.png&h=500&w=500","hex": "#002C5F"},
    {"team": "JAX", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/jax.png&h=500&w=500","hex": "#006778"},
    {"team": "KC", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/kc.png&h=500&w=500","hex": "#E31837"},
    {"team": "LAC", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lac.png&h=500&w=500","hex": "#0080C6"},
    {"team": "LA", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lar.png&h=500&w=500","hex": "#003594"},
    {"team": "MIA", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/mia.png&h=500&w=500","hex": "#008E97"},
    {"team": "MIN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/min.png&h=500&w=500","hex": "#4F2683"},
    {"team": "NE", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ne.png&h=500&w=500","hex": "#002244"},
    {"team": "NO", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/no.png&h=500&w=500","hex": "#000000"},
    {"team": "NYG", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/nyg.png&h=500&w=500","hex": "#0B2265"},
    {"team": "NYJ", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/nyj.png&h=500&w=500","hex": "#125740"},
    {"team": "LV", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lv.png&h=500&w=500","hex": "#000000"},
    {"team": "PHI", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/phi.png&h=500&w=500","hex": "#004C54"},
    {"team": "PIT", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/pit.png&h=500&w=500","hex": "#000000"},
    {"team": "SF", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/sf.png&h=500&w=500","hex": "#AA0000"},
    {"team": "SEA", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/sea.png&h=500&w=500","hex": "#002244"},
    {"team": "TB", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/tb.png&h=500&w=500","hex": "#D50A0A"},
    {"team": "TEN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ten.png&h=500&w=500","hex": "#0C2340"},
    {"team": "WAS", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/was.png&h=500&w=500","hex": "#5A1414"}
]

nfl_logo_dict = {team['team']: team['logo_url'] for team in nfl_teams}
nfl_color_dict = {team['team']: team['hex'] for team in nfl_teams}

penalty_wpa_cumulative = all_seasons_penalty_data.groupby('penalty_team', as_index=False).agg({'wpa': 'sum'})
penalty_wpa_cumulative['wpa'] = penalty_wpa_cumulative['wpa'] * 100 

penalty_wpa_cumulative['color'] = penalty_wpa_cumulative['penalty_team'].map(nfl_color_dict)

fig = px.bar(
    penalty_wpa_cumulative,
    x='penalty_team',
    y='wpa',
    color='penalty_team',
    title='Cumulative Win Probability Percentage Gained or Lost on Penalties in the Playoffs (2020-2024)',
    labels={'penalty_team': 'Team', 'wpa': 'Win Probability Added'},
    text='wpa',
    color_discrete_map=nfl_color_dict
)

fig.update_traces(
    texttemplate='%{text:.3f}',  
    textposition='inside',     
    insidetextanchor='end'     
)


for index, row in penalty_wpa_cumulative.iterrows():
    logo_url = nfl_logo_dict.get(row['penalty_team'], None)
    if logo_url:
        fig.add_layout_image(
            dict(
                source=logo_url,
                x=index,  
                y=row['wpa'] + 2,  
                xref="x",
                yref="y",
                sizex=0.9,  
                sizey=9,    
                xanchor="center",
                yanchor="bottom",
                layer="above"
            )
        )

fig.update_layout(
    yaxis_title='WPA',
    template='plotly_white',
    showlegend=False,
    annotations=[
        dict(
            text='Data source: NFLverse | By Ray Carpenter | TheSpade.substack.com',
            showarrow=False,
            x=0.5,
            y=-0.3,
            xref='paper',
            yref='paper',
            xanchor='center',
            yanchor='bottom',
            font=dict(size=10, color='gray')
        )
    ]
)

fig.show()



Processing data for the 2020 season...
Processing data for the 2021 season...
Processing data for the 2022 season...
Processing data for the 2023 season...
Processing data for the 2024 season...


In [ ]:
fig = px.bar(
    penalty_wpa_cumulative,
    x='penalty_team',
    y='wpa',
    color='penalty_team',
    title='Cumulative Win Probability Percentage Gained or Lost on Penalties in the Playoffs (2020-2024)',
    labels={'penalty_team': 'Team', 'wpa': 'Win Probability Added'},
    text='wpa',
    color_discrete_map=nfl_color_dict
)

fig.update_traces(
    texttemplate='%{text:.3f}',  
    textposition='inside',     
    insidetextanchor='end'      
)

for index, row in penalty_wpa_cumulative.iterrows():
    logo_url = nfl_logo_dict.get(row['penalty_team'], None)
    if logo_url:
        fig.add_layout_image(
            dict(
                source=logo_url,
                x=index,  
                y=row['wpa'] + 2,  
                xref="x",
                yref="y",
                sizex=20, 
                sizey=20,    
                xanchor="center",
                yanchor="bottom",
                layer="above"
            )
        )

fig.update_layout(
    yaxis_title='WPA',
    template='plotly_white',
    showlegend=False,
    annotations=[
        dict(
            text='Data source: NFLverse | By Ray Carpenter | TheSpade.substack.com',
            showarrow=False,
            x=0.5,
            y=-0.3,
            xref='paper',
            yref='paper',
            xanchor='center',
            yanchor='bottom',
            font=dict(size=10, color='gray')
        )
    ]
)

fig.show()



In [43]:
import pandas as pd
import plotly.express as px
from io import BytesIO
from PIL import Image
import requests

YEAR = 2024
url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{YEAR}.csv.gz'
data = pd.read_csv(url, compression='gzip', low_memory=False)

# NFL team logo URLs
nfl_teams = [
    {"team": "ARI", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ari.png&h=500&w=500","hex": "#97233F"},
    {"team": "ATL", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/atl.png&h=500&w=500","hex": "#a71930"},
    {"team": "BAL", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/bal.png&h=500&w=500","hex": "#241773"},
    {"team": "BUF", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/buf.png&h=500&w=500","hex": "#00338D"},
    {"team": "CAR", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/car.png&h=500&w=500","hex": "#0085CA"},
    {"team": "CHI", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/chi.png&h=500&w=500","hex": "#0B162A"},
    {"team": "CIN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/cin.png&h=500&w=500","hex": "#fb4f14"},
    {"team": "CLE", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/cle.png&h=500&w=500","hex": "#311D00"},
    {"team": "DAL", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/dal.png&h=500&w=500","hex": "#041E42"},
    {"team": "DEN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/den.png&h=500&w=500","hex": "#FB4F14"},
    {"team": "DET", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/det.png&h=500&w=500","hex": "#0076b6"},
    {"team": "GB", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/gb.png&h=500&w=500","hex": "#203731"},
    {"team": "HOU", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/hou.png&h=500&w=500","hex": "#03202f"},
    {"team": "IND", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ind.png&h=500&w=500","hex": "#002C5F"},
    {"team": "JAX", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/jax.png&h=500&w=500","hex": "#006778"},
    {"team": "KC", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/kc.png&h=500&w=500","hex": "#E31837"},
    {"team": "LAC", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lac.png&h=500&w=500","hex": "#0080C6"},
    {"team": "LA", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lar.png&h=500&w=500","hex": "#003594"},
    {"team": "MIA", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/mia.png&h=500&w=500","hex": "#008E97"},
    {"team": "MIN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/min.png&h=500&w=500","hex": "#4F2683"},
    {"team": "NE", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ne.png&h=500&w=500","hex": "#002244"},
    {"team": "NO", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/no.png&h=500&w=500","hex": "#000000"},
    {"team": "NYG", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/nyg.png&h=500&w=500","hex": "#0B2265"},
    {"team": "NYJ", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/nyj.png&h=500&w=500","hex": "#125740"},
    {"team": "LV", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lv.png&h=500&w=500","hex": "#000000"},
    {"team": "PHI", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/phi.png&h=500&w=500","hex": "#004C54"},
    {"team": "PIT", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/pit.png&h=500&w=500","hex": "#000000"},
    {"team": "SF", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/sf.png&h=500&w=500","hex": "#AA0000"},
    {"team": "SEA", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/sea.png&h=500&w=500","hex": "#002244"},
    {"team": "TB", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/tb.png&h=500&w=500","hex": "#D50A0A"},
    {"team": "TEN", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ten.png&h=500&w=500","hex": "#0C2340"},
    {"team": "WAS", "logo_url": "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/was.png&h=500&w=500","hex": "#5A1414"}
]

nfl_logo_dict = {team['team']: team['logo_url'] for team in nfl_teams}
nfl_color_dict = {team['team']: team['hex'] for team in nfl_teams}


def get_team_logo(team_abb):
    team_abb = team_abb.strip().upper()
    logo_url = nfl_logo_dict.get(team_abb, None)
    if logo_url:
        response = requests.get(logo_url)
        if response.status_code == 200:
            return Image.open(BytesIO(response.content))
    return None


data_filtered = data[(data['down'] == 4) & (data['play_type'].isin(['run', 'pass']))]

wpa_by_team = data_filtered.groupby('posteam', as_index=False).agg({'wpa': 'sum'})
wpa_by_team['wpa'] = wpa_by_team['wpa'] * 100 
wpa_by_team['color'] = wpa_by_team['posteam'].map(nfl_color_dict)


fig = px.bar(
    wpa_by_team,
    x='posteam',
    y='wpa',
    color='posteam',
    title='Total Win Probability Percentage Gained or Lost on 4th Down Offensive & Defensive Plays, 2024-25 Season',
    labels={'posteam': 'Team', 'wpa': 'Win Probability Added'},
    text='wpa',
    color_discrete_map=nfl_color_dict
)

fig.update_traces(
    texttemplate='%{text:.3f}',  
    textposition='inside',    
    insidetextanchor='end'   
)
y_min = wpa_by_team['wpa'].min() - 30 
y_max = wpa_by_team['wpa'].max() + 30

fig.update_layout(
    yaxis=dict(
    title='Win Probability Added',
    range=[y_min, y_max]
),
    xaxis_title='Team',
    yaxis_title='WPA',
    template='plotly_white',
    annotations=[
        dict(
            text='Data source: NFLverse | By Ray Carpenter | TheSpade.substack.com',
            showarrow=False,
            x=0.5,
            y=-0.3,
            xref='paper',
            yref='paper',
            xanchor='center',
            yanchor='bottom',
            font=dict(size=10, color='gray')
        )
    ]
)


def get_team_logo_base64(team_abb):
    team_abb = team_abb.strip().upper()
    logo_url = nfl_logo_dict.get(team_abb, None)
    if logo_url:
        response = requests.get(logo_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            buffered = BytesIO()
            image.save(buffered, format="PNG")
            return f"data:image/png;base64,{base64.b64encode(buffered.getvalue()).decode()}"
    return None

for index, row in wpa_by_team.iterrows():
    logo_base64 = get_team_logo_base64(row['posteam'])
    if logo_base64:
        fig.add_layout_image(
            dict(
                source=logo_base64,
                x=index,  
                y=row['wpa'] + 2,  
                xref="x",
                yref="y",
                sizex=30, 
                sizey=30, 
                xanchor="center",
                yanchor="bottom",
                layer="above"
            )
        )


custom_logo_path = "2.png"  
try:
    custom_logo = Image.open(custom_logo_path)
    buffered = BytesIO()
    custom_logo.save(buffered, format="PNG")
    custom_logo_base64 = f"data:image/png;base64,{base64.b64encode(buffered.getvalue()).decode()}"
    fig.add_layout_image(
        dict(
            source=custom_logo_base64,
            x=1.0,
            y=-0.1,
            xref="paper",
            yref="paper",
            sizex=0.3,
            sizey=0.3,
            xanchor="right",
            yanchor="bottom",
            layer="above"
        )
    )
except FileNotFoundError:
    print("2.png file not found. Skipping logo addition.")
fig.update_layout(showlegend=False)
fig.show()
